<a href="https://colab.research.google.com/github/michelstoc/Url2Pdf_api/blob/main/url2pdf_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update 
!apt-get install -y xvfb libfontconfig wkhtmltopdf


**Sur Windows**, l'installation de wkhtmltopdf nécessite le téléchargement et l'installation manuels de l'exécutable. Voici comment vous pouvez faire cela :

Allez sur la page de téléchargement de wkhtmltopdf à l'adresse suivante : https://wkhtmltopdf.org/downloads.html
Sous "Microsoft Windows", cliquez sur le lien "64-bit" ou "32-bit" selon votre système d'exploitation.
Téléchargez le fichier .exe et suivez les instructions d'installation.
Après avoir installé wkhtmltopdf, vous devez vous assurer que l'exécutable est dans votre PATH. Pour vérifier cela, ouvrez une nouvelle invite de commande et tapez wkhtmltopdf. Si l'installation a réussi, vous devriez voir une sortie indiquant la version de wkhtmltopdf. Sinon, vous devrez ajouter le chemin de l'exécutable à votre PATH.

In [ ]:
!pip install requests pdfkit beautifulsoup4 flask


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from flask import Flask, request, send_file
from bs4 import BeautifulSoup
import urllib.parse
import requests
import pdfkit

In [ ]:
app = Flask(__name__)

@app.route('/url_to_pdf', methods=['GET'])
def url_to_pdf():
    url = request.args.get('url')
    if not url:
        return "URL parameter is missing. Please provide an URL.", 400

    # Fetch HTML content
    response = requests.get(url)
    html_content = response.text

    # Parse HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Search all img elements
    for img in soup.find_all('img'):
        # Check if src attribute exists
        if 'src' in img.attrs:
            # Convert relative path to absolute
            img['src'] = urllib.parse.urljoin(url, img['src'])

    # Search all a (link) elements
    for a in soup.find_all('a'):
        # Check if href attribute exists
        if 'href' in a.attrs:
            # Convert relative path to absolute
            a['href'] = urllib.parse.urljoin(url, a['href'])

    # Convert the soup back into a string
    html_content = str(soup)

    # Convert to PDF
    path_wkhtmltopdf = '/usr/bin/wkhtmltopdf'
    config = pdfkit.configuration(wkhtmltopdf=path_wkhtmltopdf)

    try:
        pdf = pdfkit.from_string(html_content, False, configuration=config)
        return send_file(pdf, mimetype='application/pdf')
    except Exception as e:
        return str(e), 500

if __name__ == "__main__":
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
